In [1]:
import random
import torch

#### 生成数据集 
y= **Xw** + b + c(噪声满足正态分布) 
+ _n个样本数据集，2个特征_
+ w=[2. -3.4] b=4

In [ ]:
def create_data(n,w,b):
    X=torch.normal(0,1,(n,len(w)))
    y=torch.matmul(w,X)+b
    y=y+torch.normal(0,0.01,y.shape)
    return X,y.reshape((-1,1))

x,y=create_data(1000,torch.tensor([2,-3.4]),4)
print(x,'\n',y)

In [3]:
torch.normal?

Docstring:
normal(mean, std, *, generator=None, out=None) -> Tensor

Returns a tensor of random numbers drawn from separate normal distributions
whose mean and standard deviation are given.

The :attr:`mean` is a tensor with the mean of
each output element's normal distribution

The :attr:`std` is a tensor with the standard deviation of
each output element's normal distribution

The shapes of :attr:`mean` and :attr:`std` don't need to match, but the
total number of elements in each tensor need to be the same.

.. note:: When the shapes do not match, the shape of :attr:`mean`
          is used as the shape for the returned output tensor

.. note:: When :attr:`std` is a CUDA tensor, this function synchronizes
          its device with the CPU.

Args:
    mean (Tensor): the tensor of per-element means
    std (Tensor): the tensor of per-element standard deviations

Keyword args:
    generator (:class:`torch.Generator`, optional): a pseudorandom number generator for sampling
    out (Tenso